In [1]:
import osmnx as ox
import pandas as pd
import folium
import networkx as nx
import json

pd.options.display.max_columns = None

/home/sebastian/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


### Extraer en formato geoDataFrame

In [2]:
place = 'Santiago de Cali, Colombia'
tags = {'amenity': 'police'}
gdf = ox.geometries_from_place(place, tags=tags)
gdf.head()

/home/sebastian/anaconda3/lib/python3.9/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
/home/sebastian/anaconda3/lib/python3.9/site-packages/osmnx/utils_geo.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
/home/sebastian/anaconda3/lib/python3.9/site-packages/osmnx/utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


,unique_id,osmid,element_type,addr:street,amenity,name,source,geometry,phone,nodes,building,addr:city,operator,note,operator:wikidata,operator:wikipedia,ways,landuse,type
0,node/761835610,761835610,node,Carrera 1,police,ESTACIÓN DE POLICÍA DE SAN FRANCISCO,Kaart Ground Survey 2018,POINT (-76.51594 3.46439),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node/1500804323,1500804323,node,NaN,police,NaN,NaN,POINT (-76.54622 3.44684),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node/1806388190,1806388190,node,NaN,police,NaN,NaN,POINT (-76.51481 3.40093),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node/2534138410,2534138410,node,NaN,police,CAI Los Alamos,NaN,POINT (-76.50939 3.48149),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node/2705590933,2705590933,node,NaN,police,CAI,NaN,POINT (-76.50075 3.46330),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
type(gdf)

geopandas.geodataframe.GeoDataFrame

In [5]:
print(gdf.shape)

(45, 19)


In [6]:
gdf.columns

Index(['unique_id', 'osmid', 'element_type', 'addr:street', 'amenity', 'name',
       'source', 'geometry', 'phone', 'nodes', 'building', 'addr:city',
       'operator', 'note', 'operator:wikidata', 'operator:wikipedia', 'ways',
       'landuse', 'type'],
      dtype='object')

In [7]:
gdf.isnull().mean()

unique_id             0.000000
osmid                 0.000000
element_type          0.000000
addr:street           0.777778
amenity               0.000000
name                  0.088889
source                0.822222
geometry              0.000000
phone                 0.955556
nodes                 0.377778
building              0.533333
addr:city             0.955556
operator              0.888889
note                  0.977778
operator:wikidata     0.977778
operator:wikipedia    0.977778
ways                  0.866667
landuse               0.888889
type                  0.866667
dtype: float64

In [25]:
gdf["name"] = gdf["name"].fillna("")
gdf["name"] = gdf["name"].str.lower()

In [156]:
gdf.head(2)

,unique_id,osmid,element_type,addr:street,amenity,name,source,geometry,phone,nodes,building,addr:city,operator,note,operator:wikidata,operator:wikipedia,ways,landuse,type
0,node/761835610,761835610,node,Carrera 1,police,estación de policía de san francisco,Kaart Ground Survey 2018,POINT (-76.51594 3.46439),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node/1500804323,1500804323,node,NaN,police,,NaN,POINT (-76.54622 3.44684),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
import geopandas as gpd

In [29]:
m = folium.Map(location=[3.4517923, -76.5324943], zoom_start=10, tiles='CartoDB positron')
for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(f"nombre: {r['name']}; tipo: {r['element_type']}").add_to(geo_j)
    geo_j.add_to(m)
m

### Extraer en formato Grafos

In [ ]:
ox.settings.log_console=True
ox.settings.use_cache=True

In [63]:
# define the start and end locations in latlng
start_latlng = (3.4468427, -76.5462164)
end_latlng = (3.4469341,-76.5416809)

In [145]:
# location where you want to find your route
place = 'Santiago de Cali, Colombia'

# find shortest route based on the mode of travel
mode = 'drive'        # 'drive', 'bike', 'walk'

# create graph from OSM within the boundaries of some 
# geocodable place(s)
graph = ox.graph_from_place(place, network_type = mode)

2023-02-18 16:46:02 Retrieved response from cache file "cache/e1d6acdd0deb04e43637174f516730a74f0b5934.json"
2023-02-18 16:46:02 Created GeoDataFrame with 1 rows from 1 queries
2023-02-18 16:46:02 Constructed place geometry polygon(s) to query API
2023-02-18 16:46:02 Projected GeoDataFrame to +proj=utm +zone=18 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2023-02-18 16:46:02 Projected GeoDataFrame to epsg:4326
2023-02-18 16:46:02 Projected GeoDataFrame to +proj=utm +zone=18 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2023-02-18 16:46:02 Projected GeoDataFrame to epsg:4326
2023-02-18 16:46:02 Requesting data within polygon from API in 1 request(s)


/home/sebastian/anaconda3/lib/python3.9/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
/home/sebastian/anaconda3/lib/python3.9/site-packages/osmnx/utils_geo.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:


2023-02-18 16:46:02 Retrieved response from cache file "cache/9362e71e778e3a1ecc31e5c5293c1163da431f26.json"
2023-02-18 16:46:02 Got all network data within polygon from API in 1 request(s)
2023-02-18 16:46:02 Creating graph from downloaded OSM data...
2023-02-18 16:46:03 Created graph with 60501 nodes and 113802 edges
2023-02-18 16:46:05 Added edge lengths to graph
2023-02-18 16:46:05 Identifying all nodes that lie outside the polygon...
2023-02-18 16:46:07 Created nodes GeoDataFrame from graph
2023-02-18 16:46:07 Created r-tree spatial index for 60501 geometries


/home/sebastian/anaconda3/lib/python3.9/site-packages/osmnx/utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


2023-02-18 16:46:08 Identified 59243 geometries inside polygon
2023-02-18 16:46:09 Removed 1258 nodes outside polygon
2023-02-18 16:46:09 Truncated graph by polygon
2023-02-18 16:46:09 Begin topologically simplifying the graph...
2023-02-18 16:46:10 Identified 21333 edge endpoints
2023-02-18 16:46:14 Simplified graph: 59243 to 21333 nodes, 111413 to 53692 edges
2023-02-18 16:46:15 Identifying all nodes that lie outside the polygon...
2023-02-18 16:46:15 Created nodes GeoDataFrame from graph
2023-02-18 16:46:15 Created r-tree spatial index for 21333 geometries


/home/sebastian/anaconda3/lib/python3.9/site-packages/osmnx/utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


2023-02-18 16:46:16 Identified 20488 geometries inside polygon
2023-02-18 16:46:16 Removed 845 nodes outside polygon
2023-02-18 16:46:17 Removed 5 isolated nodes
2023-02-18 16:46:18 Got largest weakly connected component (20472 of 20483 total nodes)
2023-02-18 16:46:18 Truncated graph by polygon
2023-02-18 16:46:18 Counted undirected street segments incident to each node
2023-02-18 16:46:18 graph_from_polygon returned graph with 20472 nodes and 51825 edges
2023-02-18 16:46:18 graph_from_place returned graph with 20472 nodes and 51825 edges


In [127]:
#ox.folium.plot_graph_folium(graph)

In [146]:
graph = ox.add_edge_speeds(graph)
graph = ox.add_edge_travel_times(graph)

2023-02-18 16:46:28 Created edges GeoDataFrame from graph
2023-02-18 16:46:34 Created edges GeoDataFrame from graph


In [147]:
# find the nearest node to the start location
orig_node = ox.distance.get_nearest_nodes(graph,
                                          [start_latlng[1]],
                                          [start_latlng[0]])[0]
# find the nearest node to the end location
dest_node = ox.distance.get_nearest_nodes(graph,
                                          [end_latlng[1]],
                                          [end_latlng[0]])[0]

2023-02-18 16:46:34 Found nearest node (2925352959) to point (3.4468427, -76.5462164)
2023-02-18 16:46:34 Found nearest nodes to 1 points
2023-02-18 16:46:34 Found nearest node (359892939) to point (3.4469341, -76.5416809)
2023-02-18 16:46:34 Found nearest nodes to 1 points


In [148]:
print(f"node={orig_node}")
for edge in graph.out_edges(orig_node, data=True):
    print("\n=== Edge ====")
    print("Source and target node ID:", edge[:2])
    edge_attributes = edge[2]
    # remove geometry object from output
    edge_attributes_wo_geometry = {i:edge_attributes[i] for i in edge_attributes if i!='geometry'}
    print("Edge attributes:", json.dumps(edge_attributes_wo_geometry, indent=4))    

node=2925352959

=== Edge ====
Source and target node ID: (2925352959, 2925352982)
Edge attributes: {
    "osmid": 137126076,
    "oneway": true,
    "lanes": "1",
    "name": "Carrera 4 Oeste",
    "highway": "tertiary",
    "maxspeed": "30",
    "length": 64.155,
    "speed_kph": 30.0,
    "travel_time": 7.7
}

=== Edge ====
Source and target node ID: (2925352959, 2156100196)
Edge attributes: {
    "osmid": 534155253,
    "oneway": false,
    "lanes": "2",
    "name": "Carrera 4 Oeste",
    "highway": "tertiary",
    "maxspeed": "30",
    "length": 59.319,
    "speed_kph": 30.0,
    "travel_time": 7.1
}


In [149]:
#  find the shortest path
# find shortest path based on distance or time
optimizer = 'time'        # 'length','time'
shortest_route = nx.shortest_path(graph,
                                  orig_node,
                                  dest_node,
                                  weight=optimizer, method="bellman-ford")

In [150]:
shortest_route_map = ox.plot_route_folium(graph, shortest_route, 
                                          tiles='openstreetmap')
start_marker = folium.Marker(
            location = start_latlng,
            popup = "start",
            icon = folium.Icon(color='green'))
end_marker = folium.Marker(
            location = end_latlng,
            popup = "end",
            icon = folium.Icon(color='red'))
# add the circle marker to the map
start_marker.add_to(shortest_route_map)
end_marker.add_to(shortest_route_map)

shortest_route_map

2023-02-18 16:46:51 Created edges GeoDataFrame from graph


In [151]:
edge_length = ox.utils_graph.get_route_edge_attributes(
    graph, shortest_route, 'length') 

route_total_length = sum(edge_length) / 1000
print("Total route length in km:", route_total_length)

Total route length in km: 1.245085


In [154]:
speed_km_h = 15
edge_travel_time = ox.utils_graph.get_route_edge_attributes(
    graph, shortest_route, 'travel_time') 

route_total_time = sum(edge_travel_time)
print("Total route time in minutes:", route_total_length*60/speed_km_h)

Total route time in minutes: 4.98034
